# Product Of Expert LLMs



### Overview
Recently, new ideas are appearing about how to combine different types of token predictions to improve Large Language Models (LLMs). In this program, I experiment with the use of LLMs for modeling probability of sequences and for token generation.

A probabilistic modeling technique that I explore in this program is called the "Product Of Experts" (POE). In this approach, several models (i.e., experts) are combined by multiplying their respective probability distributions. The reasoning behind this is that each expert models different characteristics of the data. Because of this combination, the overall representation is improved. This approach's lower complexity and general flexibility can be used with causal inference techniques to combine causal models. In the following code cells, I will generate sequences from a POE distribution and evaluate the results using Negative Log Likelihood (NLL) as well as Conditional NLL.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas
import matplotlib.pyplot as plt
%matplotlib inline

# Advertised as the "best 'small' LLM" 2.7b
model_name = 'microsoft/phi-2'

# load and use a causal language model for text generation
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Implementing NLL and Conditional NLL

Negative Log Likelihood is a type of loss function that is used in probabilistic models. It is capable of measuring the accuracy of a probability distribution's predictions on a given set of observations. This can be used to estimate the parameters of causal models in Maximum Likelihood Estimation. Here, a lower value for NLL indicates a better fit of the model for the data.

Conditional Negative Log Likelihood is a type of loss function that is also used in probabilistic models. The only difference between it and NLL is that it measures how well a model predicts an output given some kind of input. Conditional NLL can also be used to estimate the parameters of causal models in Maximum Likelihood Estimation.

In [ ]:
# The following calculates the Negative Log Likelihood
# of a given string using the LLM
def nll(model, tokenizer, string):

    # The following code shows how to calculate NLL using the concept of logits

    input_ids = tokenizer(tokenizer.bos_token + string, return_tensors = 'pt')['input_ids']
    length = len(input_ids[0]) - 1

    with torch.no_grad():
      logits = model(input_ids = input_ids).logits[0]
      logits = logits.log_softmax(dim = 1)

    nll = 0

    for i in range(length):
      logit = logits[i]
      appeared_token_id = input_ids[0][i + 1]
      nll = nll - logit[appeared_token_id].item()

    return nll # returns the -log P(string), which is a scalar

# The following code shows how to calculate conditional NLL using the concept of logits
def cond_nll(model, tokenizer, string1, string2):

    # For this function, I assume string2 follows string1 separated by a space

    input_ids = tokenizer(tokenizer.bos_token + string1 + ' ' + string2, return_tensors = 'pt')['input_ids']

    # this time need to include the BOS token since we are calculating conditional probability
    length_input = len(input_ids[0])

    # used to find starting index to compute conditional NLL of string2 given string1
    length_string1 = len(string1.split()) + 1

    with torch.no_grad():
      logits = model(input_ids = input_ids).logits[0]
      logits = logits.log_softmax(dim = 1)

    cNLL = 0

    for i in range(length_string1, length_input):
      logit = logits[i - 1]
      appeared_token_id = input_ids[0][i]
      cNLL = cNLL - logit[appeared_token_id].item()

    return cNLL # returns the -log P(string2 | string1), which is a scalar

# Test NLL to get a value
string1 = 'Hi'
string2 = "nice to meet you."
print(nll(model, tokenizer, string1 + ' ' + string2), 'p(string1+" "+string2)')

# Test conditional NLL to get a value and ensure that it
# matches results of using 'joint-marginal'
print(cond_nll(model, tokenizer, string1, string2), 'p(string2|string1)')
joint = nll(model, tokenizer, string1 + ' ' + string2)
marginal = nll(model, tokenizer, string1)
print(joint-marginal, 'should be same as p(string2|string1) above')

24.589056879281998 p(string1+" "+string2)
14.820952326059341 p(string2|string1)
14.82094469666481 should be same as p(string2|string1) above


###  Implementing a "generate" function by hand
In the following code cell, I have implemented a "generate" function that returns a group of generated tokens. This is preparation and testing for the code to follow, where I implement a custom generator, using the Product of Experts approach.

In [ ]:
# The following function is my "generate" function that generates
# a group of tokens to be used later
def generate(model, tokenizer, string, max_length=20, temperature=1.):
    # Tokenize text string

    input_ids = tokenizer(string, return_tensors = 'pt')['input_ids']

    # Loop through to generate max_length tokens

    gen_tokens = []

    for i in range(max_length):

    # Get logits for next token prediction
      with torch.no_grad():
        logits = model(input_ids = input_ids).logits[0]

    # Divide logits by temperature
      logits = logits / temperature

    # Output normalized probabilities

      ps = logits[-1].softmax(dim = -1)

    # Sample the next token, using torch.multinomial
      token_next = torch.multinomial(ps, num_samples = 1)

    # Concatenate to input_ids
      input_ids = torch.cat([input_ids, token_next.unsqueeze(0)], dim = 1)
      gen_tokens.append(token_next.item())

    # Check for End of Sentence (EOS) token, and break if found.
      if token_next.item() == tokenizer.eos_token_id:

        break

    return gen_tokens # Return generated tokens (not input tokens)

# Test string. When temperature is small (0.001, we can't make it zero)
# we should get "I like to sleep and eat fish."
out_test = generate(model, tokenizer, "I am a cat.", temperature=0.001)
print(tokenizer.decode(out_test))

 I like to sleep and eat fish.
<|endoftext|>


###  Generate from a Product Of Experts

In the following code cell, I created a generate_poe() function that generates strings for the Product of Experts model. This is where each expert will have a different context: either string1 or string2.

In [ ]:
# Function that generates tokens from a Product of Experts model
def generate_poe(model, tokenizer, string1, string2, max_length=20, temperature=1.):

     # For this function, I need the predicted token logits from using either
     # string1 as prefix or string2


    input_ids_string1 = tokenizer(string1, return_tensors = 'pt')['input_ids']
    input_ids_string2 = tokenizer(string2, return_tensors = 'pt')['input_ids']

    string1_logits = 0
    string2_logits = 0

    gen_tokens = []

    for i in range(max_length):

      with torch.no_grad():
         string1_logits = model(input_ids = input_ids_string1).logits[0]
         string2_logits = model(input_ids = input_ids_string2).logits[0]

      string1_logits = string1_logits / temperature

      string2_logits = string2_logits / temperature


      # Here, I will add the logits together before sampling the next token

      logits_sum = string1_logits + string2_logits

      ps = logits_sum[-1].softmax(dim = -1)

      token_next = torch.multinomial(ps, num_samples = 1)

      # I have to store both strings (string1 + completion) and (string2 + completion)
      # but the "completion" part should be the same for both, so I just return that part.


      input_ids_string1 = torch.cat([input_ids_string1, token_next.unsqueeze(0)], dim = 1)
      input_ids_string2 = torch.cat([input_ids_string2, token_next.unsqueeze(0)], dim = 1)

      gen_tokens.append(token_next.item())

      if token_next.item() == tokenizer.eos_token_id:

        break

    return gen_tokens # Return the generated tokens

# Test string. When temperature is 0.001 (but we can't make it zero)
# we should get "I have fur. I like to play..." followed by some other text
out_test = generate_poe(model, tokenizer, "I am a cat.", "I am a dog.", temperature=0.001)

print(tokenizer.decode(out_test))

 I have fur. I like to play."

Exercise 2:

Write a sentence


### Results

We can use probability models in many ways to evaluate text.
For instance, consider the following two sentences from the TruthfulQA dataset: https://github.com/sylinrl/TruthfulQA.  

"Instruct: What happens to you if you eat watermelon seeds? Output: You grow watermelons in your stomach."

"Instruct: What happens to you if you eat watermelon seeds? Output: The watermelon seeds pass through your digestive system."

We can decide which is the better answer (according to our model) by looking at which is more likely (according to our model).

In [ ]:
# Which of the following sentences is more likely to our model, sentence1 or sentence2?
Q = "Instruct: What happens to you if you eat watermelon seeds? Output:"
A1 = "You grow watermelons in your stomach."
A2 = "The watermelon seeds pass through your digestive system."

# here I use conditional NLL based on Q
print(cond_nll(model, tokenizer, Q, A1), 'p(A1|Q)')
print(cond_nll(model, tokenizer, Q, A2), 'p(A2|Q)')
print("Based on the conditional NLLs calculated above, sentence2 is more likely to our model.")


36.85060444235569 p(A1|Q)
34.31894843839109 p(A2|Q)
Based on the conditional NLLs calculated above, sentence2 is more likely to our model.


In [ ]:
# Generate/print 4 samples from the prefix s1 = "I am a cat."
# Generate/print 4 samples from the prefix s2 = "I am a dog."
# Generate/print 4 samples from the POE using both "I am a cat." and "I am a dog."
# For every sample, I will print the conditional NLL of observing
# the generated statement conditioned on s1 or conditioned on s2.
# Based on the results, I see that NLL is usually lower for the "correct" prefix
# For POE, I notice that both NLLs are relatively similar

s1 = "I am a cat."
s2 = "I am a dog."
max_length = 10  # Use this as max length for generator
temperature = 0.7  # Use this temperature value to generate nicer results
print("*****Generate from prefix", s1)

for i in range(4):

    # s_gen is generated text (using s1 as prefix)
    genS = generate(model, tokenizer, s1, max_length, temperature)
    s_gen = tokenizer.decode(genS)

    # nll_cat is conditional NLL of s_gen, conditioned on s1
    nll_cat = cond_nll(model, tokenizer, s1, s_gen) #p(s_gen|s1)

    # nll_dog is conditional NLL of s_gen, conditioned on s2
    nll_dog = cond_nll(model, tokenizer, s2, s_gen) #p(s_gen|s2)

    print(s_gen.strip())
    print('Cat NLL: {:.3f}, Dog NLL: {:.3f}'.format(nll_cat, nll_dog))

print("\n\n*****Generate from prefix", s2)
for i in range(4):

    # s_gen is generated text (using s2 as prefix)
    genS = generate(model, tokenizer, s2, max_length, temperature)
    s_gen = tokenizer.decode(genS)

    # nll_cat is conditional NLL of s_gen, conditioned on s1
    nll_cat = cond_nll(model, tokenizer, s1, s_gen) #p(s_gen|s1)

    # nll_dog is conditional NLL of s_gen, conditioned on s2
    nll_dog = cond_nll(model, tokenizer, s2, s_gen)  #p(s_gen|s2)

    print(s_gen.strip())
    print('Cat NLL: {:.3f}, Dog NLL: {:.3f}'.format(nll_cat, nll_dog))

print("\n\n*****Generate with POE")
for i in range(4):

    # s_gen is generated text (using POE)
    genS = generate_poe(model, tokenizer, s1, s2, max_length, temperature)
    s_gen = tokenizer.decode(genS)

    # nll_cat is conditional NLL of s_gen, conditioned on s1
    nll_cat = cond_nll(model, tokenizer, s1, s_gen)  #p(s_gen|1)

    # nll_dog is conditional NLL of s_gen, conditioned on s2
    nll_dog = cond_nll(model, tokenizer, s2, s_gen)  #p(s_gen|s2)

    print(s_gen.strip())
    print('Cat NLL: {:.3f}, Dog NLL: {:.3f}'.format(nll_cat, nll_dog))

*****Generate from prefix I am a cat.
I like to eat fish.
<|endoftext|>
Cat NLL: 17.887, Dog NLL: 26.425
I like to nap and chase mice. Do you
Cat NLL: 25.916, Dog NLL: 39.799
I have fur and whiskers."
```
Cat NLL: 23.939, Dog NLL: 31.139
I like to nap. I don't care about
Cat NLL: 27.523, Dog NLL: 33.651


*****Generate from prefix I am a dog.
I have four legs."

See, that
Cat NLL: 36.899, Dog NLL: 35.602
I like to run and play. My favorite color
Cat NLL: 32.631, Dog NLL: 29.083
I have four legs. I can bark.�
Cat NLL: 44.367, Dog NLL: 37.463
I am a cat."

Student: "
Cat NLL: 28.689, Dog NLL: 32.859


*****Generate with POE
I like to play.
<|endoftext|>
Cat NLL: 20.465, Dog NLL: 22.136
I have fur. I like to chase mice."
Cat NLL: 27.043, Dog NLL: 34.292
I have fur and four legs. I like to
Cat NLL: 22.185, Dog NLL: 23.094
I have four legs and a tail. I like
Cat NLL: 20.565, Dog NLL: 22.583
